<a href="https://colab.research.google.com/github/kalyans04/automatic-driver-test-tracking/blob/main/YOLO_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Nov 21 04:28:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
HOME=os.getcwd()
print(HOME)

/content


In [ ]:
SOURCE_VIDEO_PATH ='/content/drive/MyDrive/lane.mp4'

In [ ]:
 !pip install ultralytics==8.0.10
 from IPython import display
 display.clear_output()
 import ultralytics
 ultralytics.checks()

Ultralytics YOLOv8.0.10 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.9/78.2 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %cd {HOME}
# !git clone https://github.com/ifzhang/ByteTrack.git
# %cd {HOME}/ByteTrack

# # workaround related to https://github.com/roboflow/notebooks/issues/80
# !sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

# !pip3 install -q -r requirements.txt
# !python3 setup.py -q develop+
# !pip install -q cython_bbox
# !pip install -q onemetric
# # workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
# !pip install -q loguru lap thop

# from IPython import display
# display.clear_output()


# import sys
# sys.path.append(f"{HOME}/ByteTrack")


# import yolox
# print("yolox.__version__:", yolox.__version__)

In [ ]:
# from yolox.tracker.byte_tracker import BYTETracker, STrack
# from onemetric.cv.utils.iou import box_iou_batch
# from dataclasses import dataclass


# @dataclass(frozen=True)
# class BYTETrackerArgs:
#     track_thresh: float = 0.25
#     track_buffer: int = 30
#     match_thresh: float = 0.8
#     aspect_ratio_thresh: float = 3.0
#     min_box_area: float = 1.0
#     mot20: bool = False

ModuleNotFoundError: ignored

In [ ]:
!pip install supervision==0.1.0


from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.1.0


In [ ]:
from supervision.draw.color import ColorPalette
from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.video.sink import VideoSink
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.tools.line_counter import LineCounter, LineCounterAnnotator

In [ ]:
model1="yolov8x.pt"

In [ ]:
from ultralytics import YOLO
model=YOLO(model1)
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 223MB/s]

Fusing... 
YOLOv8x summary: 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [ ]:
#!yolo task=detect mode=predict model=yolov8x.pt conf=0.25 source={SOURCE_VIDEO_PATH}

In [ ]:
from supervision.video.dataclasses import VideoInfo
VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=1920, height=1080, fps=25, total_frames=1624)

In [ ]:
CLASS_NAMES_DICT=model.model.names
print(model.model.names)

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [ ]:
TARGET_VIDEO_PATH=f"{HOME}/lane_frame_detect.mp4"

In [ ]:
from supervision.video.source import get_video_frames_generator
from supervision.notebook.utils import show_frame_in_notebook
from supervision.tools.detections import Detections, BoxAnnotator
from supervision.draw.color import ColorPalette
import math
from tqdm.notebook import tqdm
from scipy.spatial import distance as dist
import numpy as np
import sys
import cv2
l1=[]
generator=get_video_frames_generator(SOURCE_VIDEO_PATH)
box_annotator=BoxAnnotator(color=ColorPalette(),thickness=4,text_thickness=4,text_scale=2)
video_info=VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
with VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    max1=sys.maxsize
    max2=sys.maxsize
    for frame in tqdm(generator, total=video_info.total_frames):
      results=model(frame)[0]
      detections=Detections(
          xyxy=results.boxes.xyxy.cpu().numpy(),
          confidence=results.boxes.conf.cpu().numpy(),
          class_id=results.boxes.cls.cpu().numpy().astype(int),
      )
      new_detections=[]
      for _,confidence,class_id,tracker_id in detections:
        # print(_,confidence,class_id,tracker_id)
        if(class_id==2 or class_id ==5 or class_id==7):
          l1=[]
          l1.append(_)
          new_detections.append(l1)
          break
      for i in new_detections:
        for j in i:
          # print(j)
          x1=int(j[0])
          y1=int(j[1])
          x3=int(j[2])
          y3=int(j[3])
          # print(x1,y1,x3,y3)
          roi_vertices = [
            ((int((x1+x3)/2)-500),y3+150),  # Bottom-left
            ((int((x1+x3)/2)-500),y1),  # Top-left
            (int((x1+x3)/2),y1),  # Top-right
            (int((x1+x3)/2),(y3+150)),  # Bottom-right
          ]
          roi_vertices1 = [
            ((int((x1+x3)/2)+500),y3+150),  # Bottom-right
            ((int((x1+x3)/2)+500),y1),  # Top-right
            (int((x1+x3)/2),y1),  # Top-left
            (int((x1+x3)/2),(y3+150)),  # Bottom-left
          ]
          # cv.circle(frame, (a, y3-70), 10, (255, 0, 255), -1)  # -1 means fill the circle
          # cv.rectangle(frame,((int((x1+x3)/2)-500),y1),(int((x1+x3)/2),(y3+150)), (255, 255, 0), 5)
          cv2.rectangle(frame,(x1,y1),(x3,y3), (0, 255, 0), 4)
          # Create an empty mask with the same dimensions as the image
          mask = np.zeros_like(frame)

          # Fill the ROI polygon with white color (255, 255, 255)
          cv2.fillPoly(mask, [np.array(roi_vertices)], (255, 255, 255))
          cv2.fillPoly(mask, [np.array(roi_vertices1)], (255, 255, 255))
          # Bitwise AND the original image and the mask to isolate the ROI
          roi_image = cv2.bitwise_and(frame, mask)
          roi_image1 = cv2.bitwise_and(frame, mask)
          gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)
          gray1 = cv2.cvtColor(roi_image1, cv2.COLOR_BGR2GRAY)
          # Apply Gaussian blur to reduce noise and enhance edges
          blurred = cv2.GaussianBlur(gray, (7, 7), 0)
          blurred1 = cv2.GaussianBlur(gray1, (7, 7), 0)
          # Perform edge detection using Canny
          edges = cv2.Canny(blurred, 100, 150)
          edges1 = cv2.Canny(blurred1, 100, 150)
          # Find lines in the image using Hough Line Transform
          lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
          lines1 = cv2.HoughLinesP(edges1, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
          for line in lines:
            x, y, x2, y2 = line[0]
            if((x>(((x1+x3)/2)-500) and x<x1 and y<(y3+150) and y>y1) and (x2>(((x1+x3)/2)-500) and x2<x1 and y2<(y3+150) and y2>y1)):
              a1,b1=(x+x2)/2,(y+y2)/2
              a2,b2=(x1+x3)/2,(y1+y3)/2
              p1=(a1,b1)
              p2=(a2,b2)
              if(math.sqrt((a2-a1)**2+(b2-b1)**2)<max1):
                max1=math.sqrt((a2-a1)**2+(b2-b1)**2)
              # if(dist.euclidean(p1,p1)<max1):
              #   max1=dist.euclidean(p1,p2)
          for line in lines1:
            x, y, x2, y2 = line[0]
            if((x<(((x1+x3)/2)+500) and x>x3 and y<(y3+150) and y>y3) and (x2<(((x1+x3)/2)+500) and x2>x3 and y2<(y3+150) and y2>y1)):
              a1,b1=(x+x2)/2,(y+y2)/2
              a2,b2=(x1+x3)/2,(y1+y3)/2
              p1=(a1,b1)
              p2=(a2,b2)
              if(math.sqrt((a2-a1)**2+(b2-b1)**2)<max2):
                max2=math.sqrt((a2-a1)**2+(b2-b1)**2)
              # if(dist.euclidean(p1,p2)<max1):
              #   max2=dist.euclidean(p1,p2)
      for i in new_detections:
        for j in i:
          # print(j)
          x1=int(j[0])
          y1=int(j[1])
          x3=int(j[2])
          y3=int(j[3])
          # print(x1,y1,x3,y3)
          roi_vertices = [
            ((int((x1+x3)/2)-500),y3+150),  # Bottom-left
            ((int((x1+x3)/2)-500),y1),  # Top-left
            (int((x1+x3)/2),y1),  # Top-right
            (int((x1+x3)/2),(y3+150)),  # Bottom-right
          ]
          roi_vertices1 = [
            ((int((x1+x3)/2)+500),y3+150),  # Bottom-right
            ((int((x1+x3)/2)+500),y1),  # Top-right
            (int((x1+x3)/2),y1),  # Top-left
            (int((x1+x3)/2),(y3+150)),  # Bottom-left
          ]
          # cv.circle(frame, (a, y3-70), 10, (255, 0, 255), -1)  # -1 means fill the circle
          # cv.rectangle(frame,((int((x1+x3)/2)-500),y1),(int((x1+x3)/2),(y3+150)), (255, 255, 0), 5)
          cv2.rectangle(frame,(x1,y1),(x3,y3), (0, 255, 0), 4)
          # Create an empty mask with the same dimensions as the image
          mask = np.zeros_like(frame)

          # Fill the ROI polygon with white color (255, 255, 255)
          cv2.fillPoly(mask, [np.array(roi_vertices)], (255, 255, 255))
          cv2.fillPoly(mask, [np.array(roi_vertices1)], (255, 255, 255))
          # Bitwise AND the original image and the mask to isolate the ROI
          roi_image = cv2.bitwise_and(frame, mask)
          roi_image1 = cv2.bitwise_and(frame, mask)
          gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)
          gray1 = cv2.cvtColor(roi_image1, cv2.COLOR_BGR2GRAY)
          # Apply Gaussian blur to reduce noise and enhance edges
          blurred = cv2.GaussianBlur(gray, (7, 7), 0)
          blurred1 = cv2.GaussianBlur(gray1, (7, 7), 0)
          # Perform edge detection using Canny
          edges = cv2.Canny(blurred, 100, 150)
          edges1 = cv2.Canny(blurred1, 100, 150)
          # Find lines in the image using Hough Line Transform
          lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
          lines1 = cv2.HoughLinesP(edges1, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)
          for line in lines:
            x, y, x2, y2 = line[0]
            if((x>(((x1+x3)/2)-500) and x<x1 and y<(y3+150) and y>y1) and (x2>(((x1+x3)/2)-500) and x2<x1 and y2<(y3+150) and y2>y1)):
              a1,b1=(x+x2)/2,(y+y2)/2
              a2,b2=(x1+x3)/2,(y1+y3)/2
              p1=(a1,b1)
              p2=(a2,b2)
              if(dist.euclidean(p1,p1)<=max1):
                cv2.line(frame, (x, y), (x2, y2), (255, 0, 0), 5)
          sink.write_frame(frame)
          # for line in lines1:
          #   x, y, x2, y2 = line[0]
          #   if((x<(((x1+x3)/2)+500) and x>x3 and y<(y3+150) and y>y3) and (x2<(((x1+x3)/2)+500) and x2>x3 and y2<(y3+150) and y2>y1)):
          #     a1,b1=(x+x2)/2,(y+y2)/2
          #     a2,b2=(x1+x3)/2,(y1+y3)/2
          #     p1=(a1,b1)
          #     p2=(a2,b2)
          #     if(dist.euclidean(p1,p2)<max2+5):
          #       cv2.line(frame, (x, y), (x2, y2), (255, 0, 0), 5)
          #       sink.write_frame(frame)
          #       break
    print(max1,max2)
      # labels = [
      #   f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
      #   for _, confidence, class_id, tracker_id
      #   in detections
      # ]
      # l1.append(detections.xyxy)
      # # print(labels)
      # for i in labels:
      #   print(i)
      #   if i.__contains__('car') or i.__contains__('truck') or i.__contains__('bus'):
      #     # print(True)
      #     frame=box_annotator.annotate(frame=frame, detections=detections,labels=labels)
      #     sink.write_frame(frame)


  0%|          | 0/1624 [00:00<?, ?it/s]

Ultralytics YOLOv8.0.10 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


KeyboardInterrupt: ignored

In [ ]:
import cv2
import numpy as np
import math
SOURCE_VIDEO_PATH1='/content/lane_frame_detect.mp4'
TARGET_VIDEO_PATH1=f"/content/lane_object_detect.mp4"
video_info1=VideoInfo.from_video_path(SOURCE_VIDEO_PATH1)
with VideoSink(TARGET_VIDEO_PATH1, video_info1) as sink:
    video = cv2.VideoCapture(SOURCE_VIDEO_PATH)
    video1 = cv2.VideoCapture(SOURCE_VIDEO_PATH1)
    while True:
        ret, image = video.read()
        ret1, image1 = video1.read()
        if not ret and not ret1:
          break
        if not ret:
          video = cv2.VideoCapture(SOURCE_VIDEO_PATH)
          continue
        if not ret1:
          video1 = cv2.VideoCapture(SOURCE_VIDEO_PATH1)
          continue
        height, width = image.shape[:2]
        roi_vertices = [
            (width//4, height),  # Bottom-left
            (width//4, height // 4),  # Top-left
            (width, height//4),  # Top-right
            (width, height),  # Bottom-right
        ]

        # Create an empty mask with the same dimensions as the image
        mask = np.zeros_like(image)

        # Fill the ROI polygon with white color (255, 255, 255)
        cv2.fillPoly(mask, [np.array(roi_vertices)], (255, 255, 255))

        # Bitwise AND the original image and the mask to isolate the ROI
        roi_image = cv2.bitwise_and(image, mask)
        gray = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to reduce noise and enhance edges
        blurred = cv2.GaussianBlur(gray, (7, 7), 0)

        # Perform edge detection using Canny
        edges = cv2.Canny(blurred, 100, 150)

        # Find lines in the image using Hough Line Transform
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)

        # Draw the detected lines on the original image
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(image1, (x1, y1), (x2, y2), (255, 0, 0), 5)
            if ((x1 > (width // 4 + 150) and y1 > (height // 4 + 75)) or (x2 > (width // 4 + 150) and y2 > (height // 4 + 75))):
              cv2.line(image1, (x1, y1), (x2, y2), (255, 0, 0), 5)
              sink.write_frame(image1)
              # x=(x1+x2)//2
              # y=(y1+y2)//2
              # l4=[]
              # l4.append(x)
              # l4.append(y)
              # if(math.dist(l6,l4)<275):
              #   cv2.line(image1, (x1, y1), (x2, y2), (255, 0, 0), 5)
              #   sink.write_frame(image1)
        # Display the result
        # Wait for a key press and then close the window
        # cv2.imshow('Lane Detection', image1)
        # while True:
        #     if cv2.waitKey(10) == ord("a"):
        #         break

    video.release()
    video1.release()
    # Close the video writer
    cv2.destroyAllWindows()